In [1]:
import pandas as pd

from SMILESX import main, inference
%matplotlib inline

Using TensorFlow backend.


#### Definition of data

In [2]:
validation_data_dir = "./validation_data/"

In [3]:
extension = '.csv'

In [4]:
data_name = 'FreeSolv' # FreeSolv, ESOL, Lipophilicity
prop_tag = ''

In [5]:
if data_name == 'FreeSolv':
    data_filename = 'FreeSolv_SAMPL'
    prop_tag = 'expt'
elif data_name == 'ESOL':
    data_filename = 'ESOL_delaney-processed'
    prop_tag = 'measured log solubility in mols per litre'
elif data_name == 'Lipophilicity':
    data_filename = 'Lipophilicity'
    prop_tag = 'exp'
else:
    data_filename = data_name
    prop_tag = prop_tag

In [6]:
sol_data = pd.read_csv(validation_data_dir+data_filename+extension)

In [7]:
sol_data.head(3)

,Unnamed: 0,iupac,smiles,expt,calc
0,0,"4-methoxy-N,N-dimethyl-benzamide",COc1ccc(C(=O)N(C)C)cc1,-11.01,-9.625
1,1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,2,3-methylbut-1-ene,C=CC(C)C,1.83,2.452


In [8]:
sol_data = sol_data[['smiles',prop_tag]]

In [9]:
sol_data.head()

,smiles,expt
0,COc1ccc(C(=O)N(C)C)cc1,-11.01
1,CS(=O)(=O)Cl,-4.87
2,C=CC(C)C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21


In [10]:
sol_data.shape

(642, 2)

#### Hyperparameters optimization with GPyOpt (Bayesian optimization)

In [11]:
dhyp_range = [int(2**itn) for itn in range(3,11)] # 
dalpha_range = [float(ialpha/10.) for ialpha in range(20,40,1)] # Adam's learning rate = 10^(-dalpha_range)

if data_name != 'Lipophilicity':
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'lrate', 'type': 'discrete', 'domain': dalpha_range}
    ]
else:
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': (1024, 1024)}, # fixed
        {'name': 'lrate', 'type': 'discrete', 'domain': (3, 3)} # fixed
    ]

In [12]:
main.Main(data=sol_data, 
          data_name=data_name, 
          data_units='', 
          bayopt_bounds=bounds, 
          k_fold_number = 3, 
          augmentation = True, 
          outdir = "./data/", 
          bayopt_n_epochs = 1,
          bayopt_n_rounds = 1, 
          bayopt_it_factor = 1, 
          bayopt_on = True, 
          lstmunits_ref = 128, 
          denseunits_ref = 16, 
          embedding_ref = 32, 
          n_gpus = 1, 
          bridge_type = 'NVLink', 
          batch_size_ref = 8,
          alpha_ref = 22, 
          patience = 50, 
          n_epochs = 100)

***SMILES_X starts...***


******
***Fold #0 initiated...***
******
***Sampling and splitting of the dataset.***

Scaler: RobustScaler(copy=True, quantile_range=(5.0, 95.0), with_centering=True,
             with_scaling=True)
Train/valid/test splits: 0.80/0.10/0.10


***Data augmentation to True***

Enumerated SMILES:
	Training set: 4285
	Validation set: 501
	Test set: 569

***Tokenization of SMILES.***

Examples of tokenized SMILES from a training set:
[[' ', 'C', 'Cl', ' '], [' ', 'Cl', 'C', ' '], [' ', 'C', 'C', 'c', '1', 'c', 'c', 'n', 'c', 'c', '1', ' '], [' ', 'C', '(', 'c', '1', 'c', 'c', 'n', 'c', 'c', '1', ')', 'C', ' '], [' ', 'c', '1', '(', 'C', 'C', ')', 'c', 'c', 'n', 'c', 'c', '1', ' ']]

Number of tokens only present in a training set: 32

Number of tokens only present in a validation set: 27
Is the validation set a subset of the training set: True
What are the tokens by which they differ: set()

Number of tokens only present in a test set: 27
Is the test set a subset o

Epoch 21/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0095 - mean_absolute_error: 0.0747 - mean_squared_error: 0.0095 - val_loss: 0.0108 - val_mean_absolute_error: 0.0798 - val_mean_squared_error: 0.0108
Epoch 22/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0089 - mean_absolute_error: 0.0718 - mean_squared_error: 0.0089 - val_loss: 0.0111 - val_mean_absolute_error: 0.0763 - val_mean_squared_error: 0.0111
Epoch 23/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0090 - mean_absolute_error: 0.0731 - mean_squared_error: 0.0090 - val_loss: 0.0088 - val_mean_absolute_error: 0.0722 - val_mean_squared_error: 0.0088
Epoch 24/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0085 - mean_absolute_error: 0.0701 - mean_squared_error: 0.0085 - val_loss: 0.0094 - val_mean_absolute_error: 0.0730 - val_mean_squared_error: 0.0094
Epoch 25/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0083 - mea

Epoch 57/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0041 - mean_absolute_error: 0.0469 - mean_squared_error: 0.0041 - val_loss: 0.0086 - val_mean_absolute_error: 0.0700 - val_mean_squared_error: 0.0086
Epoch 58/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0041 - mean_absolute_error: 0.0477 - mean_squared_error: 0.0041 - val_loss: 0.0084 - val_mean_absolute_error: 0.0687 - val_mean_squared_error: 0.0084
Epoch 59/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0044 - mean_absolute_error: 0.0494 - mean_squared_error: 0.0044 - val_loss: 0.0075 - val_mean_absolute_error: 0.0648 - val_mean_squared_error: 0.0075
Epoch 60/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0045 - mean_absolute_error: 0.0505 - mean_squared_error: 0.0045 - val_loss: 0.0084 - val_mean_absolute_error: 0.0692 - val_mean_squared_error: 0.0084
Epoch 61/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0046 - mea

Epoch 93/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0032 - mean_absolute_error: 0.0428 - mean_squared_error: 0.0032 - val_loss: 0.0076 - val_mean_absolute_error: 0.0632 - val_mean_squared_error: 0.0076
Epoch 94/100
134/134 [==============================] - 1s 10ms/step - loss: 0.0032 - mean_absolute_error: 0.0415 - mean_squared_error: 0.0032 - val_loss: 0.0077 - val_mean_absolute_error: 0.0625 - val_mean_squared_error: 0.0077
Epoch 95/100
134/134 [==============================] - 1s 9ms/step - loss: 0.0030 - mean_absolute_error: 0.0400 - mean_squared_error: 0.0030 - val_loss: 0.0090 - val_mean_absolute_error: 0.0682 - val_mean_squared_error: 0.0090
Epoch 96/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0027 - mean_absolute_error: 0.0382 - mean_squared_error: 0.0027 - val_loss: 0.0074 - val_mean_absolute_error: 0.0620 - val_mean_squared_error: 0.0074
Epoch 97/100
134/134 [==============================] - 1s 9ms/step - loss: 0.0033 - me

67/67 [==============================] - 0s 7ms/step - loss: 0.0251 - mean_absolute_error: 0.1272 - mean_squared_error: 0.0251 - val_loss: 0.0237 - val_mean_absolute_error: 0.1215 - val_mean_squared_error: 0.0237
Epoch 13/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0251 - mean_absolute_error: 0.1264 - mean_squared_error: 0.0251 - val_loss: 0.0346 - val_mean_absolute_error: 0.1447 - val_mean_squared_error: 0.0346
Epoch 14/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0265 - mean_absolute_error: 0.1284 - mean_squared_error: 0.0265 - val_loss: 0.0318 - val_mean_absolute_error: 0.1400 - val_mean_squared_error: 0.0318
Epoch 15/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0243 - mean_absolute_error: 0.1257 - mean_squared_error: 0.0243 - val_loss: 0.0231 - val_mean_absolute_error: 0.1207 - val_mean_squared_error: 0.0231
Epoch 16/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0250 - mean_absolute_error: 0.128

67/67 [==============================] - 1s 10ms/step - loss: 0.0103 - mean_absolute_error: 0.0781 - mean_squared_error: 0.0103 - val_loss: 0.0186 - val_mean_absolute_error: 0.0994 - val_mean_squared_error: 0.0186
Epoch 49/100
67/67 [==============================] - 1s 9ms/step - loss: 0.0102 - mean_absolute_error: 0.0775 - mean_squared_error: 0.0102 - val_loss: 0.0257 - val_mean_absolute_error: 0.1170 - val_mean_squared_error: 0.0257
Epoch 50/100
67/67 [==============================] - 0s 5ms/step - loss: 0.0104 - mean_absolute_error: 0.0783 - mean_squared_error: 0.0104 - val_loss: 0.0205 - val_mean_absolute_error: 0.1016 - val_mean_squared_error: 0.0205
Epoch 51/100
67/67 [==============================] - 1s 8ms/step - loss: 0.0098 - mean_absolute_error: 0.0764 - mean_squared_error: 0.0098 - val_loss: 0.0195 - val_mean_absolute_error: 0.1026 - val_mean_squared_error: 0.0195
Epoch 52/100
67/67 [==============================] - 1s 10ms/step - loss: 0.0092 - mean_absolute_error: 0.0

67/67 [==============================] - 1s 8ms/step - loss: 0.0062 - mean_absolute_error: 0.0593 - mean_squared_error: 0.0062 - val_loss: 0.0194 - val_mean_absolute_error: 0.0966 - val_mean_squared_error: 0.0194
Epoch 85/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0063 - mean_absolute_error: 0.0591 - mean_squared_error: 0.0063 - val_loss: 0.0180 - val_mean_absolute_error: 0.0915 - val_mean_squared_error: 0.0180
Epoch 86/100
67/67 [==============================] - 1s 8ms/step - loss: 0.0062 - mean_absolute_error: 0.0590 - mean_squared_error: 0.0062 - val_loss: 0.0168 - val_mean_absolute_error: 0.0899 - val_mean_squared_error: 0.0168
Epoch 87/100
67/67 [==============================] - 1s 8ms/step - loss: 0.0060 - mean_absolute_error: 0.0579 - mean_squared_error: 0.0060 - val_loss: 0.0171 - val_mean_absolute_error: 0.0887 - val_mean_squared_error: 0.0171
Epoch 88/100
67/67 [==============================] - 0s 7ms/step - loss: 0.0062 - mean_absolute_error: 0.059

273/273 [==============================] - 2s 6ms/step - loss: 0.0237 - mean_absolute_error: 0.1198 - mean_squared_error: 0.0237 - val_loss: 0.0217 - val_mean_absolute_error: 0.1142 - val_mean_squared_error: 0.0217
Epoch 6/100
273/273 [==============================] - 2s 6ms/step - loss: 0.0201 - mean_absolute_error: 0.1145 - mean_squared_error: 0.0201 - val_loss: 0.0251 - val_mean_absolute_error: 0.1256 - val_mean_squared_error: 0.0251
Epoch 7/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0174 - mean_absolute_error: 0.1045 - mean_squared_error: 0.0174 - val_loss: 0.0182 - val_mean_absolute_error: 0.0990 - val_mean_squared_error: 0.0182
Epoch 8/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0159 - mean_absolute_error: 0.0994 - mean_squared_error: 0.0159 - val_loss: 0.0230 - val_mean_absolute_error: 0.1157 - val_mean_squared_error: 0.0230
Epoch 9/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0183 - mean_absolute_error:

273/273 [==============================] - 2s 7ms/step - loss: 0.0038 - mean_absolute_error: 0.0447 - mean_squared_error: 0.0038 - val_loss: 0.0100 - val_mean_absolute_error: 0.0652 - val_mean_squared_error: 0.0100
Epoch 42/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0034 - mean_absolute_error: 0.0432 - mean_squared_error: 0.0034 - val_loss: 0.0096 - val_mean_absolute_error: 0.0631 - val_mean_squared_error: 0.0096
Epoch 43/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0029 - mean_absolute_error: 0.0392 - mean_squared_error: 0.0029 - val_loss: 0.0154 - val_mean_absolute_error: 0.0802 - val_mean_squared_error: 0.0154
Epoch 44/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0036 - mean_absolute_error: 0.0450 - mean_squared_error: 0.0036 - val_loss: 0.0108 - val_mean_absolute_error: 0.0669 - val_mean_squared_error: 0.0108
Epoch 45/100
273/273 [==============================] - 2s 9ms/step - loss: 0.0029 - mean_absolute_er

Epoch 77/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0013 - mean_absolute_error: 0.0281 - mean_squared_error: 0.0013 - val_loss: 0.0074 - val_mean_absolute_error: 0.0557 - val_mean_squared_error: 0.0074
Epoch 78/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0014 - mean_absolute_error: 0.0291 - mean_squared_error: 0.0014 - val_loss: 0.0079 - val_mean_absolute_error: 0.0566 - val_mean_squared_error: 0.0079
Epoch 79/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0012 - mean_absolute_error: 0.0266 - mean_squared_error: 0.0012 - val_loss: 0.0091 - val_mean_absolute_error: 0.0605 - val_mean_squared_error: 0.0091
Epoch 80/100
273/273 [==============================] - 2s 8ms/step - loss: 0.0014 - mean_absolute_error: 0.0289 - mean_squared_error: 0.0014 - val_loss: 0.0095 - val_mean_absolute_error: 0.0594 - val_mean_squared_error: 0.0095
Epoch 81/100
273/273 [==============================] - 2s 7ms/step - loss: 0.0014 - mea

In [13]:
pred_from_ens = inference.Inference(data_name=data_name, 
                                    smiles_list = ['CC','CCC','C=O','ABC','DEF'], 
                                    data_units = '',
                                    k_fold_number = 3,
                                    augmentation = True, 
                                    outdir = "./data/")

***SMILES_X for inference starts...***


***Checking the SMILES list for inference***

***Data augmentation.***

Enumerated SMILES: 5

***Tokenization of SMILES.***

Full vocabulary: ['pad', 'unk', '[C@@H]', 'P', 'N', '[C@H]', '#', '-', ' ', '2', 'Br', '[S+2]', '(', '/', '3', '=', '\\', '1', 'I', 'Cl', 'n', 's', 'O', 'S', '[C@@]', 'C', ')', '[O-]', '[C@]', 'c', '[N+]', '[nH]', '5', 'F', '4']
Of size: 35

Maximum length of tokenized SMILES: 51 tokens

***Inference of SMILES property done.***


In [14]:
pred_from_ens

,SMILES,ens_pred_mean,ens_pred_sd
0,CC,0.414519,0.0469825
1,CCC,0.440597,0.0342451
2,C=O,0.0125581,0.0353095
